Mounting Google Drive

In [ ]:
#Import Python Packages
#from google.colab import drive
#drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


Importing Necessary Modules

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree, metrics
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest
!pip install sklearn-genetic
!pip install sklearn-genetic-opt
from sklearn.feature_selection import chi2, f_classif
from sklearn.neural_network import MLPClassifier
import sklearn_genetic
from sklearn_genetic import GASearchCV
from sklearn_genetic import ExponentialAdapter
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler


import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Reading Data

In [ ]:
training = pd.read_csv("/gdrive/My Drive/CIS508Ass4-2/SantanderCustomerSatisfaction-TRAIN.csv")
test = pd.read_csv("/gdrive/My Drive/CIS508Ass4-2/SantanderCustomerSatisfaction-TEST-WithoutTARGET.csv")

Performing EDA on the data

In [ ]:
training.info()
print()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75818 entries, 0 to 75817
Columns: 370 entries, ID to var38
dtypes: float64(110), int64(260)
memory usage: 214.0 MB


In [ ]:
print("Training Shape", training.shape)
print("--------------")
print("Test shape", test.shape)

Training Shape (76020, 371)
--------------
Test shape (75818, 370)


In [ ]:
training.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [ ]:
#To get list of names of all Columns from a dataframe

TrainCols = list(training.columns.values)
TestCols = list(test.columns.values)
print(TrainCols)
print(TestCols)

['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1', 'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1', 'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var40_ult1', 'imp_op_var41_comer_ult1', 'imp_op_var41_comer_ult3', 'imp_op_var41_efect_ult1', 'imp_op_var41_efect_ult3', 'imp_op_var41_ult1', 'imp_op_var39_efect_ult1', 'imp_op_var39_efect_ult3', 'imp_op_var39_ult1', 'imp_sal_var16_ult1', 'ind_var1_0', 'ind_var1', 'ind_var2_0', 'ind_var2', 'ind_var5_0', 'ind_var5', 'ind_var6_0', 'ind_var6', 'ind_var8_0', 'ind_var8', 'ind_var12_0', 'ind_var12', 'ind_var13_0', 'ind_var13_corto_0', 'ind_var13_corto', 'ind_var13_largo_0', 'ind_var13_largo', 'ind_var13_medio_0', 'ind_var13_medio', 'ind_var13', 'ind_var14_0', 'ind_var14', 'ind_var17_0', 'ind_var17', 'ind_var18_0', 'ind_var18', 'ind_var19', 'ind_var20_0', 'ind_var20', 'ind_var24_0', 'ind_var24', 'ind_var25_cte', 'ind_var26_0', 'ind_var26_cte', 'ind_var26', 'ind_var25_0', 'ind_var2

In [ ]:
#Removing 'Target' column

Xtrain = training[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = training[['TARGET']].copy()
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = test.copy()

(76020, 370)
(76020, 1)


Printing the distribution of target class values

In [ ]:
training["TARGET"].value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

Performing **Feature Selection** using

*   Variance Threshold: Removing constant columns
*   Correlation matrix: Removing highly correlated columns

The objective is to provide only important columns to the classifier to avoid over-fitting.



In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size = 0.2 , random_state = 1, stratify = Ytrain)

In [ ]:
Y_test.shape

(15204, 1)

To handle the under-representation of the target class, we will perform either:


*  Undersampling
*  Oversampling





In [ ]:
# instantiating the random undersampler
rus = RandomUnderSampler(sampling_strategy= 1) 
# resampling X, y
X_rus, Y_rus = rus.fit_resample(X_train, Y_train)
# new class distribution
X_rus.shape

(4812, 370)

In [ ]:
Y_rus["TARGET"].value_counts()

0    2406
1    2406
Name: TARGET, dtype: int64

In [ ]:
Y_rus.shape

(4812, 1)

In [ ]:
X_test.shape

(15204, 370)

In [ ]:
Y_test.shape

(15204, 1)

Creating a decision tree classifier and training it
Changes made over a default classifier:


*   Splitting criterion changed to "Entropy" instead of "Gini"
*   Max number of leaf nodes changed to 20



In [ ]:
#GENETIC ALGORITHM FOR FEATURE SELECTION 25 ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV

x = X_rus
y = Y_rus

clf = DecisionTreeClassifier()
model = GeneticSelectionCV(
    clf, cv=5, verbose=0,
    scoring="accuracy", max_features=25,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(x, y)

#Do feature selection using the genetic algo
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())

GA_25 = cols

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False False False False  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False  True False False False False
 False False False False False False False False  T

In [ ]:
#GENETIC ALGORITHM FOR FEATURE SELECTION 50 ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV

x = X_rus
y = Y_rus

clf = DecisionTreeClassifier()
model = GeneticSelectionCV(
    clf, cv=5, verbose=0,
    scoring="accuracy", max_features=50,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(x, y)

#Do feature selection using the genetic algo
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())
GA_50 = cols

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False False  True False False False False False False False False False
 False False False  True  True False False False False False False False
 False False False False False False False False False False False  True
 False False False  True False False  True False False False False False
 False False False False False False False False False False False  True
 False False False False  True False False False False False False False
 False False  True False False False False False False False  True False
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False  True  True False False False False False False False False False
 False False  True False False  True False False False False False False
 False False False False  True False False False False False False False
 False False False  True False False False  True Fa

In [ ]:

#K-BEST FEATURE Selection =================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=25)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_rus, Y_rus)
SelectKBest_25 = fs.get_support(True)
print ("25 Important Features", SelectKBest_25)

25 Important Features [  2  25  30  31  32  33  34  39  50  64  89  91  97  98  99 100 105 115
 139 148 159 183 194 281 284]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 22  23  26  27  37  38  44  45  57  58  59  60  61  62  72  73  81  85
  86  92  93 103 104 110 111 121 132 133 134 135 136 137 146 147 156 162
 163 166 171 175 180 181 182 187 190 193 195 196 202 203 204 205 206 207
 214 215 216 217 218 219 221 222 223 224 227 230 231 233 235 237 238 239
 240 243 244 245 246 247 248 249 250 251 253 262 263 266 269 270 272 286
 288 304 306 307 308 309 314 315 316 317 318 319 320 321 322 328 349 350
 351 352 354 357 358 359 360 366] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature

In [ ]:

#K-BEST FEATURE Selection of 50 features=================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=50)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_rus, Y_rus)
SelectKBest_50 = fs.get_support(True)
print ("50 Important Features", SelectKBest_50)

50 Important Features [  2  13  14  16  17  25  28  30  31  32  33  34  39  49  50  64  77  80
  89  91  94  97  98  99 100 105 114 115 139 148 152 155 159 168 169 172
 177 183 191 194 225 264 281 283 284 340 341 343 344 369]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 22  23  26  27  37  38  44  45  57  58  59  60  61  62  72  73  81  85
  86  92  93 103 104 110 111 121 132 133 134 135 136 137 146 147 156 162
 163 166 171 175 180 181 182 187 190 193 195 196 202 203 204 205 206 207
 214 215 216 217 218 219 221 222 223 224 227 230 231 233 235 237 238 239
 240 243 244 245 246 247 248 249 250 251 253 262 263 266 269 270 272 286
 288 304 306 307 308 309 314 315 316 317 318 319 320 321 322 328 349 350
 351 352 354 357 358 359 360 366] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature

In [ ]:
common_features_25 = []
common_features_50 = []
for i in SelectKBest_25:
  if i in GA_25:
    common_features_25.append(i)
for i in SelectKBest_50:
  if i in GA_50:
    common_features_50.append(i)
print(common_features_25, "\n", common_features_50)


[2, 183] 
 [2, 16, 39, 64, 100, 148, 159, 191, 264, 283, 341]


In [ ]:
# Getting only common columns
X_rus_common25 = X_rus.iloc[:,[2, 183]]
X_rus_common50 = X_rus.iloc[:, [2, 16, 39, 64, 100, 148, 159, 191, 264, 283, 341]]


In [ ]:
X_rus_common25 = X_rus.iloc[:, common_features_25]
X_rus_common50 = X_rus.iloc[:, common_features_50]
X_test25 = X_test.iloc[:,common_features_25]
X_test50 = X_test.iloc[:,common_features_50]

In [ ]:
X_rus_common50

,var15,imp_op_var39_efect_ult1,ind_var13,ind_var30,num_var13_corto,num_var35,num_var42,saldo_var42,num_aport_var13_hace3,num_meses_var12_ult3,saldo_medio_var13_corto_hace2
0,63,0.0,0,1,0,6,6,120060.00,0,1,0.0
1,25,0.0,0,1,0,3,3,3.00,0,0,0.0
2,69,0.0,0,0,0,0,0,0.00,0,0,0.0
3,29,0.0,0,1,0,3,3,3420.00,0,0,0.0
4,26,0.0,0,1,0,6,3,9530.07,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4807,35,540.0,0,1,0,6,3,63.00,0,0,0.0
4808,24,150.0,0,1,0,6,3,150.00,0,0,0.0
4809,38,0.0,0,0,0,0,0,0.00,0,0,0.0
4810,47,3750.0,0,1,0,9,3,1137.69,0,0,0.0


# Random Forest Classifier with top 25 features and GA Hyperparameter tuning

In [ ]:
# Top 25 RF Classifier
clf = RandomForestClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30),
              'max_leaf_nodes': Integer(2, 35),
              'n_estimators': Integer(100, 300)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common25, Y_rus)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test25)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.727785	0.0322941  	0.753533   	0.651496   
1  	40    	0.746945	0.00771045 	0.753533   	0.717789   
2  	40    	0.75159 	0.00140835 	0.753948   	0.749377   
3  	40    	0.75214 	0.00173633 	0.753948   	0.746467   
4  	40    	0.752805	0.00107683 	0.754364   	0.750831   
5  	40    	0.752972	0.00122786 	0.75478    	0.750831   
6  	40    	0.752941	0.0010313  	0.75478    	0.751455   
7  	40    	0.75267 	0.00155135 	0.75478    	0.748753   
8  	40    	0.753086	0.00152609 	0.75478    	0.749169   
9  	40    	0.753128	0.00157236 	0.75478    	0.749169   
10 	40    	0.753232	0.00195331 	0.756027   	0.747298   
11 	40    	0.752982	0.00135203 	0.756027   	0.750416   
12 	40    	0.753377	0.00129259 	0.754988   	0.750831   
13 	40    	0.753315	0.00135075 	0.754988   	0.750416   
14 	40    	0.753813	0.00102289 	0.754988   	0.751455   
15 	40    	0.753834	0.00149781 	0.754988   	0.749169   
16 	40    	0.753512	0.00185746 	0.754988   	0.74

/usr/local/lib/python3.7/dist-packages/sklearn_genetic/genetic_search.py:555: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(self.X_, self.y_)


{'min_weight_fraction_leaf': 0.01278039135922495, 'bootstrap': True, 'max_depth': 13, 'max_leaf_nodes': 25, 'n_estimators': 261}
0.7551960010523546
Stats achieved in each generation:  {'gen': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 'fitness': [0.7277847049044056, 0.7469451371571073, 0.7515897755610973, 0.7521404821280132, 0.7528054862842892, 0.7529717373233583, 0.7529405652535328, 0.7526704073150458, 0.7530860349127181, 0.7531275976724854, 0.7532315045719036, 0.7529821280133, 0.7533769742310888, 0.753314630091438, 0.753813383208645, 0.7538341645885286, 0.7535120532003325, 0.7538757273482959, 0.7544576059850372, 0.7540108063175394, 0.7535224438902742, 0.7532211138819618, 0.7524314214463841, 0.7532730673316708, 0.7526911886949293, 0.7530756442227765], 'fitness_std': [0.03229408380542457, 0.007710452061488375, 0.0014083506036154196, 0.0017363337236329175, 0.0010768284628455032, 0.001227861284016812, 0.001031298877865256, 0.0015513477

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_weight_fraction_leaf': 0.01278039135922495, 'bootstrap': True, 'max_depth': 13, 'max_leaf_nodes': 25, 'n_estimators': 261}


In [ ]:
# kaggle_test25 = test[['var15', 'num_var5', 'num_var30']]
# kaggle_test50 = test[['var15', 'imp_op_var41_efect_ult1', 'imp_op_var39_efect_ult1',
#        'imp_op_var39_ult1', 'ind_var13_0', 'num_var13_corto_0', 'num_var13',
#        'saldo_var30', 'saldo_var42', 'var36']]

kaggle_test25 = test.iloc[:, common_features_25]
kaggle_test50 = test.iloc[:, common_features_50]

In [ ]:
# Getting only common columns

X_train_25_whole = X_train.iloc[:, common_features_25]
X_train_50_whole = X_train.iloc[:, common_features_50]

In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
clf = RandomForestClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_25_whole, Y_train)
clf_predict = clf.predict(kaggle_test25)
sample_submission = Xtest[["ID"]]
sample_submission['TARGET'] = clf_predict
sample_submission.to_csv("/gdrive/My Drive/CIS508Ass4-2/RF25.csv",index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Random Forest Classifier with top 50 features and HPT

In [ ]:
# Top 50 RF Classifier
clf = RandomForestClassifier()


param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30),
              'max_leaf_nodes': Integer(2, 35),
              'n_estimators': Integer(100, 300)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common50, Y_rus)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test50)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.701288	0.0280788  	0.752909   	0.663549   
1  	40    	0.733603	0.0203735  	0.753948   	0.682253   
2  	40    	0.748743	0.008031   	0.754988   	0.732544   
3  	40    	0.753658	0.00262899 	0.756442   	0.744805   
4  	40    	0.75479 	0.00270276 	0.757897   	0.744805   
5  	40    	0.75586 	0.00130212 	0.757897   	0.753741   
6  	40    	0.755881	0.0014489  	0.757897   	0.752078   
7  	40    	0.756099	0.0014772  	0.757897   	0.753741   
8  	40    	0.756058	0.00158984 	0.757897   	0.751663   
9  	40    	0.756608	0.00122169 	0.757897   	0.754364   
10 	40    	0.756297	0.00159774 	0.757897   	0.752494   
11 	40    	0.75612 	0.0016514  	0.757897   	0.752494   
12 	40    	0.756629	0.00130592 	0.757897   	0.753533   
13 	40    	0.756557	0.00141935 	0.757897   	0.754156   
14 	40    	0.756588	0.00141268 	0.757897   	0.753948   
15 	40    	0.756463	0.00162161 	0.757897   	0.75187    
16 	40    	0.756286	0.00140774 	0.759144   	0.75

/usr/local/lib/python3.7/dist-packages/sklearn_genetic/genetic_search.py:555: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(self.X_, self.y_)


{'min_weight_fraction_leaf': 0.010623027867919058, 'bootstrap': True, 'max_depth': 25, 'max_leaf_nodes': 16, 'n_estimators': 235}
0.7867666403578005
Stats achieved in each generation:  {'gen': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 'fitness': [0.7012884455527847, 0.7336034912718205, 0.7487427265170409, 0.7536575228595179, 0.7547901080631754, 0.7558603491271819, 0.7558811305070656, 0.7560993349958436, 0.7560577722360764, 0.7566084788029925, 0.7562967581047382, 0.7561201163757275, 0.7566292601828761, 0.7565565253532835, 0.7565876974231089, 0.756463009143807, 0.7562863674147963, 0.7558395677472983, 0.7561720698254365, 0.7563902743142145, 0.7564110556940982, 0.7563383208645054, 0.7539692435577721, 0.7551745635910224, 0.7557772236076474, 0.7560889443059018], 'fitness_std': [0.0280788204716404, 0.020373480220728216, 0.008030995109419795, 0.0026289882960712388, 0.002702758071409898, 0.0013021155609160419, 0.0014488959279761373, 0.001477

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_weight_fraction_leaf': 0.010623027867919058, 'bootstrap': True, 'max_depth': 25, 'max_leaf_nodes': 16, 'n_estimators': 235}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
clf = RandomForestClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_50_whole, Y_train)
clf_predict = clf.predict(kaggle_test50)
sample_submission = Xtest[["ID"]]
sample_submission['TARGET'] = clf_predict
sample_submission.to_csv("/gdrive/My Drive/CIS508Ass4-2/RF50.csv",index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Decision Tree Classifier top 25 features Hyperparameter tuning with GA

In [ ]:
# Top 25 RF Classifier
clf = DecisionTreeClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_samples_split' : Integer(10,100),'max_depth': Integer(1,20)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common25, Y_rus)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test25)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.735526	0.00490778 	0.748337   	0.726309   
1  	40    	0.741199	0.00452456 	0.750831   	0.73192    
2  	40    	0.743734	0.00333169 	0.746675   	0.733791   
3  	40    	0.745948	0.00266982 	0.75       	0.740441   
4  	40    	0.746893	0.00287916 	0.751663   	0.740025   
5  	40    	0.748275	0.00155943 	0.75187    	0.745012   
6  	40    	0.747984	0.00177203 	0.75187    	0.744597   
7  	40    	0.748566	0.00253575 	0.752286   	0.742311   
8  	40    	0.748639	0.00335764 	0.753117   	0.741272   
9  	40    	0.749273	0.00194003 	0.753117   	0.744805   
10 	40    	0.748971	0.00190632 	0.753117   	0.74522    
11 	40    	0.748639	0.00282082 	0.753117   	0.740025   
12 	40    	0.749075	0.00153624 	0.751039   	0.744389   
13 	40    	0.746166	0.00489686 	0.751039   	0.733167   
14 	40    	0.745542	0.00515675 	0.749792   	0.730258   
15 	40    	0.747911	0.00246719 	0.749792   	0.740856   
16 	40    	0.747911	0.00225789 	0.749792   	0.74

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_samples_split': 47, 'max_depth': 4}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DTClassifier
clf = DecisionTreeClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_25_whole, Y_train)
clf_predict = clf.predict(kaggle_test25)

sample_submission = Xtest[["ID"]]
sample_submission['TARGET'] = clf_predict
sample_submission.to_csv("/gdrive/My Drive/CIS508Ass4-2/DT25.csv",index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## Decision Tree Classifier Top 50 features hyperparameter tuning using GA

In [ ]:
# Top 50 DT Classifier
clf = DecisionTreeClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_samples_split' : Integer(10,100),'max_depth': Integer(1,20)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common50, Y_rus)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test50)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.738113	0.0157011  	0.748545   	0.683084   
1  	40    	0.745906	0.00336703 	0.750831   	0.73857    
2  	40    	0.744836	0.00465812 	0.753325   	0.735661   
3  	40    	0.747371	0.00330435 	0.753325   	0.740648   
4  	40    	0.746976	0.00305739 	0.753325   	0.73857    
5  	40    	0.747766	0.00319647 	0.75478    	0.743766   
6  	40    	0.749616	0.0021976  	0.752286   	0.744805   
7  	40    	0.749512	0.00228527 	0.751455   	0.743766   
8  	40    	0.749813	0.00305981 	0.754572   	0.739817   
9  	40    	0.75    	0.00240472 	0.754572   	0.74335    
10 	40    	0.749065	0.0036165  	0.752078   	0.737116   
11 	40    	0.750644	0.00151988 	0.754364   	0.747714   
12 	40    	0.749855	0.00246336 	0.754364   	0.745636   
13 	40    	0.749408	0.00312984 	0.752702   	0.741272   
14 	40    	0.748691	0.00331468 	0.752702   	0.742103   
15 	40    	0.749574	0.00312515 	0.752702   	0.742934   
16 	40    	0.749408	0.00333687 	0.752702   	0.74

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_samples_split': 95, 'max_depth': 7}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
clf = DecisionTreeClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_50_whole, Y_train)
clf_predict = clf.predict(kaggle_test50)

sample_submission = Xtest[["ID"]]
sample_submission['TARGET'] = clf_predict
sample_submission.to_csv("/gdrive/My Drive/CIS508Ass4-2/DT50.csv",index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## MLP Classifier with top 25 features and GA

In [ ]:
import itertools

In [ ]:
# # Top 25 MLP Classifier


param_grid = {'tol': Continuous(1e-2, 1e10, distribution='log-uniform'),
              'alpha': Continuous(1e-5, 2e-5),
              'activation': Categorical(['logistic', 'tanh']),
              'batch_size': Integer(300, 350)
             }
# The base classifier to tune
clf = MLPClassifier(hidden_layer_sizes=(50, 30))
# Our cross-validation strategy (it could be just an int)
cv = StratifiedKFold(n_splits=3, shuffle=True)
# The main class from sklearn-genetic-opt
evolved_estimator = GASearchCV(estimator=clf,
                              cv=cv,
                              scoring='accuracy',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True,
                              population_size=20,
                              generations=25)
# Train and optimize the estimator
evolved_estimator.fit(X_rus_common25, Y_rus)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test25)
print(accuracy_score(Y_test, y_predict_ga))

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.678034	0.0312041  	0.732128   	0.627805   
1  	40    	0.700707	0.0142005  	0.732128   	0.665628   
2  	40    	0.712271	0.00736215 	0.732128   	0.698878   
3  	40    	0.710276	0.0106644  	0.732128   	0.693267   
4  	40    	0.712365	0.00874274 	0.732128   	0.698462   
5  	40    	0.7093  	0.00918628 	0.719867   	0.689111   
6  	40    	0.715752	0.00537214 	0.721945   	0.704281   
7  	40    	0.717955	0.00786229 	0.732751   	0.703865   
8  	40    	0.717062	0.00702844 	0.724439   	0.6968     
9  	40    	0.718153	0.00622813 	0.724439   	0.699917   
10 	40    	0.718288	0.0098308  	0.73857    	0.697007   
11 	40    	0.726787	0.00756628 	0.73857    	0.715919   
12 	40    	0.724595	0.0103957  	0.73857    	0.705112   
13 	40    	0.722839	0.00766946 	0.73857    	0.708229   
14 	40    	0.724304	0.00848106 	0.73857    	0.701372   
15 	40    	0.724075	0.00663016 	0.73857    	0.714879   
16 	40    	0.72473 	0.00998294 	0.738986   	0.70

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'tol': 0.01331725042468669, 'alpha': 1.7133964093634217e-05, 'activation': 'tanh', 'batch_size': 304}
0.8008418837148119


In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'tol': 0.01331725042468669, 'alpha': 1.7133964093634217e-05, 'activation': 'tanh', 'batch_size': 304}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the MLPClassifier
clf = MLPClassifier(**tuned_params)
clf.fit(X_train_25_whole, Y_train)
clf_predict = clf.predict(kaggle_test25)

sample_submission = Xtest[["ID"]]
sample_submission['TARGET'] = clf_predict
sample_submission.to_csv("/gdrive/My Drive/CIS508Ass4-2/MLP25.csv",index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## MLP Classifier with top 50 features and GA

In [ ]:
# # Top 50 LP Classifier



param_grid = {'tol': Continuous(1e-2, 1e10, distribution='log-uniform'),
              'alpha': Continuous(1e-5, 2e-5),
              'activation': Categorical(['logistic', 'tanh']),
              'batch_size': Integer(300, 350)
             }
# The base classifier to tune
clf = MLPClassifier(hidden_layer_sizes=(50, 30))
# Our cross-validation strategy (it could be just an int)
cv = StratifiedKFold(n_splits=3, shuffle=True)
# The main class from sklearn-genetic-opt
evolved_estimator = GASearchCV(estimator=clf,
                              cv=cv,
                              scoring='accuracy',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True,
                              population_size=20,
                              generations=25)
# Train and optimize the estimator
evolved_estimator.fit(X_rus_common50, Y_rus)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test50)
print(accuracy_score(Y_test, y_predict_ga))

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.697257	0.00329213 	0.70345    	0.690981   
1  	40    	0.699938	0.00197139 	0.705528   	0.697215   
2  	40    	0.700488	0.00211815 	0.705528   	0.696384   
3  	40    	0.701985	0.00322779 	0.705943   	0.6968     
4  	40    	0.706047	0.00576379 	0.717165   	0.699086   
5  	40    	0.706297	0.00498324 	0.717165   	0.701579   
6  	40    	0.707949	0.00552538 	0.717165   	0.701995   
7  	40    	0.70824 	0.00600703 	0.717165   	0.699293   
8  	40    	0.705943	0.00565849 	0.716542   	0.697423   
9  	40    	0.709185	0.00609975 	0.716542   	0.69867    
10 	40    	0.710037	0.00594947 	0.716542   	0.698462   
11 	40    	0.709404	0.00703114 	0.716542   	0.696592   
12 	40    	0.709601	0.00603033 	0.716542   	0.700333   
13 	40    	0.71036 	0.00637683 	0.716542   	0.697839   
14 	40    	0.709549	0.00638948 	0.716542   	0.697839   
15 	40    	0.708209	0.00729272 	0.716542   	0.698047   
16 	40    	0.710256	0.00743436 	0.716542   	0.69

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'tol': 2024.121909978109, 'alpha': 1.4359948108079394e-05, 'activation': 'tanh', 'batch_size': 302}
0.6858063667455933


In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'tol': 2024.121909978109, 'alpha': 1.4359948108079394e-05, 'activation': 'tanh', 'batch_size': 302}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the MLPClassifier
clf = MLPClassifier(**tuned_params)
clf.fit(X_train_50_whole, Y_train)
clf_predict = clf.predict(kaggle_test50)

sample_submission = Xtest[["ID"]]
sample_submission['TARGET'] = clf_predict
sample_submission.to_csv("/gdrive/My Drive/CIS508Ass4-2/MLP50.csv",index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
